In [1]:
import pandas as pd
import numpy as np
from shutil import rmtree # Удаление директориии во всем её содержимым
import sys # Для работы с параметрами для запуска программы
from os import listdir, makedirs # Для работы с директориями
from os.path import isfile, join, exists # Для работы с файлами

def process_store(sell, supply, inventory, sell_parser):
    '''
    На вход 3 файла - продажи, закупки, инвентарь.
    На выход - фреймы данных, которые далее будут переводиться в csv
    '''
    # Устанавливаем индексирование, удаляем пустые строки
    supply.set_index('date', inplace=True) 
    inventory.set_index('date', inplace=True)
    
    ''' 
    Идентификационный код мы заменяем либо на 'a', либо на 'p' - все остальное нам не нужно
    Чтобы понять, какую букву взять - используем sell_parser - 8 элемент идентификационного кода
    '''
    sell.sku_num = sell.sku_num.map(sell_parser)

    # Подсчитываем количество яблок и ручек, проданных за один день
    def apple(daily): 
        return np.sum(daily == 'a')
    def pen(daily): 
        return np.sum(daily == 'p')
    
    '''
    Группирует по дате. По данным столбца 'sku_num' рассчитывается количество
    проданных за один день ручек и яблок. Происходит подразделение на 2 столбца:
    'apple', 'pen' - со значениями соответствующих количеств
    '''
    sells_daily = sell.groupby('date')['sku_num'].agg([apple, pen])
    '''
    Закупки за день = { Закупки за месяц, если день == 1 или 15
                      { 0               , иначе
    
    Изменения за день = Закупки за день - Продажи за день
    '''
    changes_daily = supply.reindex_like(sells_daily).fillna(0) - sells_daily
    
    '''
    Теперь суммируем значения за каждый день в месяцах, группируем по месяцам,
    а не по дням
    
    '2011-12-25'[:-2] = '2011-12-'
    '''
    monthly_cumsum_changes = changes_daily.groupby(lambda date: date[:-2]).agg(np.cumsum)
    extend_inventory = inventory.reindex_like(monthly_cumsum_changes).shift(1).fillna(axis=0, method='ffill').fillna(0)
    store_daily_state = extend_inventory + monthly_cumsum_changes
    stolen_monthly = store_daily_state.reindex_like(inventory) - inventory
    
    return store_daily_state, stolen_monthly, sells_daily.groupby(lambda date: date[:-2]).sum()

# Группирует по году и штату и суммирует месячные данные
def agregate_statistics(stats):
    return pd.concat(stats).groupby(['year', 'state']).sum()


In [2]:
# Названия двух директорий: входные данные и ответы
inp_dir = 'input'
out_dir = 'output'

In [3]:
# Берется 7-ой символ строки transaction - он содержит информацию о том, продается яблоко или ручка
sell_parser = lambda transaction: transaction[6]

# Составляем список всех файлов из директории в аргументе
all_files = [f for f in listdir(inp_dir) if isfile(join(inp_dir, f))] 

# Из этого списка файлов берем только префиксы типа "MS-s1", "MS-b2" и т.п. - по одному разу 
stores = [f[:-8] for f in all_files if f[-8:] == "sell.csv"] 

# Папки нет - создаем её, чтобы избежать ошибок
if not exists(out_dir): 
    makedirs(out_dir)
else:
    rmtree(out_dir)
    makedirs(out_dir)

    
statistics = []
for store in stores:
    '''
    Каждый шаг цикла соответствует работе с одним префиксом - store. С помощью этого префикса мы получаем доступ к
    соответствующим файлам о закупках, продажах, инвентаре.
    '''
    # Печать для того, чтобы пользователь понимал, на какой стадии выполнения находится программа
    print ("Processing \"{0}\"".format(store[:-min(len(store), 1)]))

   
    # Если на этапе чтения файла возникла ошибка, то выбрасывается исключение и работа продолжается
    try:
        inventory = pd.read_csv(inp_dir + '/' + store + 'inventory.csv') # input/MS-b1-inventory.csv
        sold = pd.read_csv(inp_dir + '/' + store + 'sell.csv')
        supply = pd.read_csv(inp_dir + '/' + store + 'supply.csv')
    except:
        print ("Error while reading \"{0}\" files, ignore".format(store[:-min(len(store), 1)]))
        continue

    state, stolen, sells = process_store(sold, supply, inventory, sell_parser)


    # Осталось создать csv файлы в директории для ответов
    state.to_csv(out_dir + '/' + store + 'daily.csv')  # Состояние склада на каждый день
    stolen.to_csv(out_dir + '/' + store + 'steal.csv') # Месячные данные о количестве сворованного товара

    sells.index = stolen.index # Копируем индексы
    
    # Формируем заданное в условии отображение данных
    stats = sells.join(stolen, lsuffix='_sold', rsuffix='_stolen').reset_index()
    stats['state'] = store[:2] # Штат - первые 2 символа суффикса "MS-b1-"
    stats['test'] = store[:6]
    stats['year'] = stats.date.map(lambda date: date[:4]) # Данные только по годам
    # del stats['date'] - я эту строку убрал и вроде ничего не поменялось
    statistics.append(stats) 

# Переводим список в дата фрейм, перемещаем его по указанному пути
agregate_statistics(statistics).to_csv(out_dir + '/states.csv') # Агрегированные данные о продажах и сворованном
    
print('Finished successfully')


Processing "MS-b1"
Processing "MS-b2"
Processing "MS-m1"
Processing "MS-m2"
Processing "MS-s1"
Processing "MS-s2"
Processing "MS-s3"
Processing "MS-s4"
Processing "MS-s5"
Finished successfully


In [4]:
st = pd.read_csv("output/states.csv")
st

,year,state,apple_sold,pen_sold,apple_stolen,pen_stolen
0,2006,MS,2152006,155633,418.0,461.0
1,2007,MS,2150384,154730,377.0,346.0
2,2008,MS,2163559,154597,383.0,382.0
3,2009,MS,2152502,155409,433.0,454.0
4,2010,MS,2149787,155523,418.0,441.0
5,2011,MS,2154860,154158,436.0,452.0
6,2012,MS,2160040,155798,381.0,421.0
7,2013,MS,2157901,154496,361.0,444.0
8,2014,MS,2153434,154687,433.0,441.0
9,2015,MS,2152497,153562,370.0,395.0
